# Tracing Without LangChain

LangSmith lets you instrument **any LLM application,** no LangChain required. This can help you debug, evaluate, and monitor your app without having to learn any particular framework's unique semantics.

The basic tracing functionality works by posting `run` messages to the LangSmith platform. Runs are structured log messages organized within call hierarchy for each trace. Though you can manually create runs using the `Client` or REST API, LangSmith provides two recommended ways to instrument your application in python (there currently is just the `RunTree` for js):

1. Using a `RunTree`
2. Using the `@traceable` decorator, which manages the RunTree lifecycle on your behalf.

In the following walkthrough, you will create an example app using the `@traceable` decorator and explore some configurations you can do for logging auxiliary information.

The example chat app used in this walkthrough generates an argument about a topic. It is composed of three separate components, which will all show up in the traces:

1. An argument generation function
2. Critique function
3. Refine function

Each of these "chain" functions will call an openai `llm` function, and the whole series of calls will itself be organized beneath a parent `argument_chain` function.

Put together, the program will generate a trace like the following:

![RunTree 1](img/snapshot_1.png)

Now that we know what we're building, let's get started!

## Prerequisites

Let's first install some required packages. We'll need both the langsmith SDK and the OpenAI SDK.

In [1]:
%pip install -U langsmith > /dev/null
%pip install -U openai > /dev/null

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Next, configure the API Key in the environment to make sure traces are logged to your account.

In [2]:
%env LANGCHAIN_API_KEY=<your-api-key>
%env LANGCHAIN_PROJECT=tracing-cookbook-tutorial

## Using the decorator

Next, define your chat application. Use the `@traceable` decorator to automatically instrument your
function calls.

The decorator works by creating a run tree for you each time the function is called. The function inputs, name, and other information is added to the tree and streamed to LangSmith. If the function raises an error or if it returns a response, that information is also added to the tree, and updates are sent to LangSmith. This is all done on a background thread to avoid blocking your app's execution.

The app below combines three `chain` runs, which each call an openai `llm` function. The `llm` run type corresponds to any LLM or chat model call. We recommend that you type all other general functions as `chain` runs, as these are the most general-purpose.

In [1]:
from datetime import datetime
from typing import List, Optional, Tuple

import openai
from langsmith.run_helpers import traceable


# We will label this function as an "llm" call to organize and render it better in the web app
@traceable(run_type="llm")
def call_openai(data: List[dict], model: str = "gpt-3.5-turbo", temperature: float = 0.0):
    return openai.ChatCompletion.create(
        model=model,
        messages=data,
        temperature=temperature,
    )


# 'Chain' run_type's can correspond to any function and are the most common traced run
@traceable(run_type="chain")
def argument_generator(query: str, additional_description: str = "") -> str:
    return call_openai(
        [
            {"role": "system", "content": f"You are a debater making an argument on a topic."
             f"{additional_description}"
             f" The current time is {datetime.now()}"},
            {"role": "user", "content": f"The discussion topic is {query}"}
        ]
    ).choices[0].message.content


@traceable(run_type="chain")
def critic(argument: str) -> str:
    return call_openai(
        [ 
            {"role": "system", "content": f"You are a critic."
           "\nWhat unresolved questions or criticism do you have after reading the following argument?"
            "Provide a concise summary of your feedback."},
            {"role": "system", "content": argument}
            
        ]
    ).choices[0].message.content


@traceable(run_type="chain")
def refiner(query: str, additional_description: str, current_arg: str, criticism: str) -> str:
    return call_openai(
        [
            {"role": "system", "content": f"You are a debater making an argument on a topic."
             f"{additional_description}"
             f" The current time is {datetime.now()}"},
            {"role": "user", "content": f"The discussion topic is {query}"},
            {"role": "assistant", "content": current_arg},
            {"role": "user", "content": criticism},
            {"role": "system", "content": "Please generate a new argument that incorporates the feedback from the user."}
        ]
    ).choices[0].message.content
    

@traceable(run_type="chain")      
def argument_chain(query: str, additional_description: str = "") -> str:
    argument = argument_generator(query, additional_description)
    criticism = critic(argument)
    return refiner(query, additional_description, argument, criticism)

Now call the chain. If you set up your API key correctly at the start of this notebook, all the results should be traced to  [LangSmith](https://www.smith.langchain.com).

In [2]:
result = argument_chain(
    "Whether sunshine is good for you.", 
    additional_description="Provide a concise, few sentence argument on why sunshine is good for you.",
)
print(result)

Sunshine, in moderation, is good for you because it provides essential vitamin D, which is crucial for bone health, immune function, and mental well-being. However, it is important to acknowledge the potential risks of excessive sun exposure, such as skin damage and an increased risk of skin cancer. By practicing sun safety measures, such as wearing sunscreen and seeking shade during peak hours, individuals can enjoy the benefits of sunshine while minimizing the potential drawbacks. Additionally, while there is anecdotal evidence suggesting that sunlight can improve mood and sleep patterns, further research is needed to fully understand and establish the causal relationship between sunlight and mental well-being.


## Working with runs

The above is all you need to save your app traces to LangSmith! The decorator handles the call relationships for you in a background thread to avoid interfering with your program execution. You can try adding exceptions or other information to the functions to see how those are shown in the app.

Beyond tracing, you may want to use LangSmith for other things like monitoring user feedback. You can view the run information by looking at the `RunTree` created for that event. To do so, change any wrapped function's signature to accept a `run_tree` argument. Then the `@traceable` decorator will inject the current run tree object into the wrapped function. This can be useful if you want to:
- Add user feedback to the run
- Inspect or save the run ID or its parent
- Manually log child runs or their information to another destination
- Explicitly pass the run tree to other functions that may be called within thread or process pools (or on separate machines) to maintain trace cohesion

Below, our `argument_chain2` function is identical to the previous one but views the injected run_tree so we can fetch the latest run.

In [3]:
from langsmith import RunTree
    
@traceable(run_type="chain")      
def argument_chain2(query: str, *, additional_description: str = "", run_tree: RunTree) -> str:
    argument = argument_generator(query, additional_description)
    criticism = critic(argument)
    refined_argument = refiner(query, additional_description, argument, criticism)       
    return refined_argument, run_tree.id

In [4]:
result, run_id = argument_chain2(
    "Whether sunshine is good for you.", 
    additional_description="Provide a concise, few sentence argument on why sunshine is good for you.",
)

View the run ID that you've collected and then create a public link to share. You could also add the ID to other logs or instrumentation in your app.

In [5]:
from langsmith import Client

client = Client()

In [6]:
client.create_feedback(
    run_id,
    "user_feedback",
    score=0.5,
    correction={"generation": "Sunshine is nice. Full stop."},
)

Feedback(id=UUID('a27e9cb3-e42a-4f29-8aca-68c851c30f6d'), created_at=datetime.datetime(2023, 8, 9, 4, 55, 29, 179158), modified_at=datetime.datetime(2023, 8, 9, 4, 55, 29, 179164), run_id=UUID('ae0e4090-300f-478d-913d-dcaf03ce8448'), key='user_feedback', score=0.5, value=None, comment=None, correction={'generation': 'Sunshine is nice. Full stop.'}, feedback_source=FeedbackSourceBase(type='api', metadata=None))

In [8]:
# You can use run_id's to create shared links
shared_link = client.share_run(run_id)
shared_link

'https://smith.langchain.com/public/7e7b6caa-80bc-41af-bed1-35cd1011de77/r'

[![RunTree 2](./img/snapshot_2.png)]('https://smith.langchain.com/public/7e7b6caa-80bc-41af-bed1-35cd1011de77/r')

## Configuring Traces

One way to make your application traces more useful or actionable is to tag or add metadata to the logs. That way you can do things like track the version of your code or deployment environment in a single project.

The traceable decorator can be configured to add additional information such as:
- string tags
- arbitrary key-value metadata
- custom trace names
- manually-specified run ID

Below is an example. 

In [9]:
# You can add tags and metadata (or even the project name) directly in the decorator
@traceable(run_type="chain", name="My Argument Chain", tags=["tutorial"], metadata={"githash": "e38f04c83"})      
def argument_chain_3(query: str, additional_description: str = "") -> str:
    argument = argument_generator(query, additional_description)
    criticism = critic(argument)
    return refiner(query, additional_description, argument, criticism)       

In [10]:
from uuid import uuid4

requested_uuid = uuid4()

result = argument_chain_3(
    "Whether sunshine is good for you.", 
    additional_description="Provide a concise, few sentence argument on why sunshine is good for you.",
    # You can also add tags, metadata, or the run ID directly via arguments to the langsmith_extra argument
    # at all-time.
    langsmith_extra={
        "tags": ["another-tag"],
        "metadata": {"another-key": 1},
        "run_id": requested_uuid,
    }
)

In [11]:
# We can log feedback for the run directly since we've controlled the ID it's assuming
client.create_feedback(
    requested_uuid,
    "user_feedback",
    score=1,
    source_info={"origin": "example notebook"}

)

Feedback(id=UUID('43d50b3d-a341-4863-a102-b7e8af0524c7'), created_at=datetime.datetime(2023, 8, 9, 4, 57, 12, 43727), modified_at=datetime.datetime(2023, 8, 9, 4, 57, 12, 43732), run_id=UUID('7830a736-566a-496c-8d5d-2f1f4ad56ed9'), key='user_feedback', score=1.0, value=None, comment=None, correction=None, feedback_source=FeedbackSourceBase(type='api', metadata={'origin': 'example notebook'}))

Now you can navigate to the run with the requested UUID to see the simulated "user feedback".

[![Tagged Run Tree](./img/snapshot_3.png)](https://dev.smith.langchain.com/public/502a9bc4-44b1-4e86-bb19-3d6b9766f248/r)

Clicking in to the 'Metadata' tab, you can see the metadata has been stored for the trace.

[![Run Tree Metadata](./img/snapshot_3_metadata.png)](https://dev.smith.langchain.com/public/502a9bc4-44b1-4e86-bb19-3d6b9766f248/r?tab=2)

Once you've stored these tagged runs, you can filter and search right in the web app by clicking on the suggested filters or by writing out the query in the search bar:


![Filtering Tags](./img/snapshot_3_tag_filter.png)

## Recap

In this walkthrough, you made an example LLM application and instrumented it using the `@traceable` decorator from the LangSmith SDK.

You also added tags and metadata and even logged feedback to the runs. The traceable decorator is a light-weight and flexible way to start debugging and monitoring your application.